# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
import datetime
from datetime import datetime, timedelta
import re

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')
world_cities = pd.read_csv('/kaggle/input/world-cities/worldcities.csv')
cost_living = pd.read_csv('/kaggle/input/cost-of-living-index-by-country/Cost_of_living_index.csv')

**Посмотрим на данные**

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.info()

In [ ]:
sample_submission.head(5)

In [ ]:
#Для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

In [ ]:
data.City.value_counts()

In [ ]:
#замена наименований городов
world_cities.loc[world_cities.city_ascii == 'Porto', 'city_ascii'] = 'Oporto'
world_cities.loc[world_cities.city_ascii == 'Zürich', 'city_ascii'] = 'Zurich' 
world_cities.loc[world_cities.city_ascii == 'Kraków', 'city_ascii'] = 'Krakow'

In [ ]:
# из дополнительного датасета по населению городов выбираем только нужные нам города Европы
world_cities = world_cities[(world_cities.city_ascii.isin(data.City.unique()))&
                  (world_cities.country != 'United States') &
                  (world_cities.country != 'Canada') &
                  (world_cities.country != 'Venezuela') &
                  (world_cities.country != 'Philippines') &
                  (world_cities.country != 'Colombia') &
                  (world_cities.country != 'Brazil')]
# удаляем ненужные столбцы
world_cities = world_cities.drop(['city', 'lat','lng','country','iso2','iso3','admin_name','id','capital'], axis=1)

world_cities

In [ ]:
# в датасете уровня жизни городов обрезаем в колоне City название города
cost_living = cost_living[cost_living.City != 'London, Canada']
cost_living.City = cost_living.City.apply(lambda x:x.split(',')[0])
cost_living.loc[cost_living.City == 'Krakow (Cracow)', 'City'] = 'Krakow'
# из дополнительного датасета по населению городов выбираем только нужные нам города Европы
cost_living = cost_living[(cost_living.City.isin(data.City.unique()))] 
cost_living

Подробнее по признакам:

* City: Город
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com'
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data.sample(5)

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Cleaning and Prepping Data


## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 


In [ ]:
data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')
data['Number_of_Reviews_isNAN']

In [ ]:
data['Cuisine Style_NAN'] = pd.isna(data['Cuisine Style']).astype('uint8')
data['Cuisine Style_NAN']

In [ ]:
data.sample(5)

In [ ]:
data['Number of Reviews'].fillna(0)

In [ ]:
data['Cuisine Style'].fillna("['nameless']") 

In [ ]:
data['Price Range'].fillna('no_data')

In [ ]:
data.nunique(dropna=False)

In [ ]:
data.info()

## 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [ ]:
data = pd.get_dummies(data, columns=['City'], dummy_na=True)

In [ ]:
data['Cuisine Style'] = data['Cuisine Style'].fillna('Other')                       
data['Cuisine Style'] = data['Cuisine Style'].apply(lambda x: str(x).strip('[]'))   
data['Cuisine Style'] = data['Cuisine Style'].apply(lambda x:                       
                                                str(x).replace("'",""))         
data['Cuisine Style'] = data['Cuisine Style'].apply(lambda x: x.split(", "))        
cuisine_style = data['Cuisine Style'].explode()                                   
group = pd.get_dummies(cuisine_style)
group = group.reset_index()
group = group.groupby(['index']).sum()
data = data.join(group)
data['Cuisine Style'] = data['Cuisine Style'].apply(lambda x: len(x)) 

In [ ]:
data.replace({'Price Range': {'no_data':0, '$':1, '$$ - $$$':2, '$$$$':3}})

# EDA 

### Посмотрим распределение признака

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

In [ ]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [ ]:
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Здесь мы можем наблюдать, что хоть и  на первый взгляд распределение экспотенциально, но если брать города по отдельности, то распределение становится, нормальным, просто в больших городах больше ресторанов, и из-за этого мы имеем смещение.
Можно попробовать провести нормализацию признака и посмотреть, что будет 

### Посмотрим распределение целевой переменной

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

In [ ]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

## Корреляция признаков 

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

Практически у всех признаков слабая корреляция, в целом мы на верном пути, так как 

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [ ]:
# Объединяем датасеты с населением городов и уровнем жизни
world_cities = world_cities.join(cost_living.set_index('City'), on='city_ascii')
# заполняем пропуски средним значением по столбцу
world_cities['Cost of Living Index'].fillna(world_cities['Cost of Living Index'].mean(), inplace=True)
world_cities['Restaurant Price Index'].fillna(world_cities['Restaurant Price Index'].mean(), inplace=True)
world_cities['Local Purchasing Power Index'].fillna(world_cities['Local Purchasing Power Index'].mean(), inplace=True)
world_cities['Rent Index'].fillna(world_cities['Rent Index'].mean(), inplace=True)
world_cities['Cost of Living Plus Rent Index'].fillna(world_cities['Cost of Living Plus Rent Index'].mean(), inplace=True)
world_cities['Groceries Index'].fillna(world_cities['Groceries Index'].mean(), inplace=True) 
world_cities['Rank'].fillna(0, inplace=True)

In [ ]:
# объединяем получившийся датасет с основными данными
data = data.join(world_cities.set_index('city_ascii'), on='City') 
data

In [ ]:
def preproc_data(df_input):
    # функция ранжировки цен - замена символов на числа 1,2,3,0
    def fill_Price(range):
        if range == '$':
            return 1
        elif range == '$$ - $$$':
            return 2
        elif range == '$$$$':
            return 3
    
    df_output = df_input.copy()
    
    #  1. Предобработка  
    # убираем не нужные для модели признаки
    df_output.drop(['Restaurant_id','ID_TA',], axis = 1, inplace=True)
    
    
    #  2. NAN  
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    df_output['Number_of_Reviews_isNAN'] = pd.isna(df_output['Number of Reviews']).astype('uint8')
    df_output['Number of Reviews'].fillna(0, inplace=True)
    
    # тут ваш код по обработке NAN
    df_output['Price Range_isNAN'] = pd.isna(df_output['Price Range']).astype('uint8')
    df_output['Price Range'].fillna(data['Price Range'].mode()[0], inplace=True)
    df_output['Reviews_isNAN'] = pd.isna(df_output['Reviews']).astype('uint8')
    df_output['Reviews'].fillna('[[], []]',inplace=True)

    
    
    #  3. Encoding  
    european = ['London', 'Paris', 'Berlin',  'Prague',  'Vienna', 'Amsterdam',
            'Brussels', 'Hamburg', 'Munich',  'Budapest', 'Warsaw', 
           'Dublin',  'Edinburgh', 'Zurich',  'Geneva', 'Krakow','Bratislava', 'Luxembourg', 'Ljubljana']
    mediter = ['Madrid', 'Barcelona', 'Milan', 'Rome', 'Lyon','Athens','Oporto','Lisbon']
    scandinavian = ['Stockholm', 'Copenhagen',  'Oslo', 'Helsinki']

    def local(x):
        if x in european:
            return 'Europe'
        elif x in mediter:
            return 'Mediterr'
        elif x in scandinavian:
            return 'Scand'
    
    df_output['local_rest'] = df_output.City.apply(local)
    
    df_output = pd.get_dummies(df_output, columns=[ 'City',], dummy_na=True)

    
    # Cuisine Style
    # заполняем пропуски значением nameless (без имени)
    df_output['Cuisine_Style_isNAN'] = pd.isna(df_output['Cuisine Style']).astype('uint8')
    df_output['Cuisine'] = df_output['Cuisine Style'].fillna("['nameless']")
    # т.к. значения в столбце представлены в виде строки, то разбиваем ее на список
    df_output['Cuisine'] = df_output['Cuisine'].apply(lambda x: eval(x))
                                     
    
    #df_output = pd.get_dummies(df_output, columns=[ 'Cuisine',], dummy_na=True)
    # добавляем новый признак - количество кухонь, которые представляет ресторан
    df_output['Count cuisine'] = df_output['Cuisine'].apply(lambda x: len(x))
    
    
    
    #  4. Feature Engineering 
    # тут ваш код не генерацию новых фитчей
    # Price Range
    df_output['Price Range'] = df_output['Price Range'].apply(lambda x: fill_Price(x))
    
     
    
    # вытащим из столбца с отзывами дату
    df_output['Reviews data'] = df_output['Reviews'].apply(lambda x: re.findall(r"\d\d\/\d\d\/\d{4}", str(x)))
    df_output[['Date_1','Date_2']]=df_output['Reviews data'].apply(pd.Series,1)
    df_output['Date_1']=pd.to_datetime(df_output['Date_1'])
    df_output['Date_2']=pd.to_datetime(df_output['Date_2'])
    # вычислим разницу между датами
    df_output['Delta review']=abs(df_output['Date_1']-df_output['Date_2'])
    # приведем разницу в дни
    df_output['Delta review']=df_output['Delta review'].dt.days
    # пропуски заполняем средним значением
    df_output['Delta review'].fillna(round(df_output['Delta review'].mean()),inplace=True)
    
    
    #  5. Clean  
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    df_output.drop(['Date_1','Date_2'], axis = 1, inplace=True)
    
    return df_output

### Проверим что получилось

In [ ]:
df_preproc = preproc_data(data)
df_preproc


In [ ]:
df_preproc.info()

In [ ]:
# Признаки по уровню жизни хорошо скоррелированны с остальными
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(df_preproc.drop(['sample'], axis=1).corr(),)

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наша цель
X = train_data.drop(['Rating'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model

In [ ]:
# Импортируем библиотеки
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

# Округлим предсказанные значения до степени округления целевой переменной
def round(prediction):
        if prediction < 0.25:
            return 0
        elif 0.25 < prediction <= 0.75:
            return 0.5
        elif 0.75 < prediction <= 1.25:
            return 1
        elif 1.25 <prediction <= 1.75:
            return 1.5
        elif 1.75 < prediction <= 2.25:
            return 2
        elif 2.25 < prediction <= 2.75:
            return 2.5
        elif 2.75 < prediction <= 3.25:
            return 3
        elif 3.25 < prediction <= 3.75:
            return 3.5
        elif 3.75 < prediction <= 4.25:
            return 4
        elif 4.25 < prediction <= 4.75:
            return 4.5
        else:
            return 5
        
for i in range(y_pred.size):
        y_pred[i]=round(y_pred[i])

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)
for i in range(predict_submission.size):
        predict_submission[i]=round(predict_submission[i])

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)